1b. Code Implementation for Integrating MySQL with Python
---

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import plotly.express as px
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from matplotlib import pyplot as plt
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn import metrics


In [2]:
# !pip install sqlalchemy

# Uncomment above and run cell to install package for the first time

In [3]:
query = """
    SELECT * FROM churn_status;
"""

In [4]:
import pandas as pd
from sqlalchemy import create_engine

ENDPOINT = 'heicoders-playground.c2ced10ceyki.ap-southeast-1.rds.amazonaws.com'
PORT = 3306
USERNAME = 'student300'
PASSWORD = 'heicoders_AI300'
DBNAME = 'ai300_capstone'

database_conn = create_engine(f'mysql+pymysql://{USERNAME}:{PASSWORD}@{ENDPOINT}/{DBNAME}')

df = pd.read_sql(query, database_conn)
df

,customer_id,status,churn_label,churn_category,churn_reason
0,0002-ORFBO,Stayed,No,,
1,0003-MKNFE,Stayed,No,,
2,0004-TLHLJ,Churned,Yes,Competitor,Competitor had better devices
3,0011-IGKFF,Churned,Yes,Dissatisfaction,Product dissatisfaction
4,0013-EXCHZ,Churned,Yes,Dissatisfaction,Network reliability
...,...,...,...,...,...
7038,9987-LUTYD,Stayed,No,,
7039,9992-RRAMN,Churned,Yes,Dissatisfaction,Product dissatisfaction
7040,9992-UJOEL,Joined,No,,
7041,9993-LHIEB,Stayed,No,,


In [5]:
another_query = """
    SELECT cs.customer_id, cs.churn_label, id.*, acc.*, ac_use.*,c.*
FROM churn_status AS cs
LEFT JOIN customer AS id
ON cs.customer_id = id.customer_id
LEFT JOIN account AS acc
ON cs.customer_id = acc.customer_id
LEFT JOIN account_usage AS ac_use
ON acc.account_id = ac_use.account_id
LEFT JOIN city AS c
ON id.zip_code = c.zip_code
WHERE churn_label IN ('Yes','No');
"""
df = pd.read_sql(another_query, database_conn)
df

,customer_id,churn_label,customer_id,gender,age,senior_citizen,married,num_dependents,zip_code,account_id,...,stream_music,total_monthly_fee,total_charges_quarter,total_refunds,area_id,zip_code,city,latitutde,longitude,population
0,0013-EXCHZ,Yes,0013-EXCHZ,Female,75,Yes,Yes,0,93010,BFIN-DLMOA,...,No,83.90,267.40,0.00,607,93010,Camarillo,34.227846,-119.079903,42853
1,0014-BMAQU,No,0014-BMAQU,Male,52,No,Yes,0,94558,AFEO-XOOCP,...,No,84.65,5377.80,0.00,963,94558,Napa,38.489789,-122.270110,63947
2,0016-QLJIS,No,0016-QLJIS,Female,43,No,Yes,1,95681,DEMQ-MFXWC,...,Yes,90.45,5957.90,0.00,1390,95681,Sheridan,38.984756,-121.345074,1219
3,0019-EFAEP,No,0019-EFAEP,Female,32,No,No,0,91942,AIPP-VTDXJ,...,No,101.30,7261.25,0.00,303,91942,La Mesa,32.782501,-117.016110,24005
4,0019-GFNTW,No,0019-GFNTW,Female,39,No,No,0,93441,CJHA-SRKIB,...,No,45.05,2560.10,0.00,716,93441,Los Olivos,34.704340,-120.026090,1317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6986,9848-JQJTX,No,9848-JQJTX,Male,52,No,No,0,90291,ZTCJ-AYASC,...,Yes,100.90,7459.05,0.00,94,90291,Venice,33.991782,-118.479229,31021
6987,9921-ZVRHG,Yes,9921-ZVRHG,Female,44,No,No,0,93673,WKQW-QPWUM,...,No,80.45,1072.00,0.00,819,93673,Traver,36.456091,-119.486225,646
6988,9938-TKDGL,No,9938-TKDGL,Male,38,No,Yes,3,93544,VKPX-MVNBC,...,Yes,99.55,6668.00,0.00,755,93544,Llano,34.500091,-117.765862,1220
6989,9938-ZREHM,No,9938-ZREHM,Female,41,No,Yes,0,93033,PQGJ-VYMBE,...,Yes,72.10,2658.40,16.36,615,93033,Oxnard,34.156629,-119.117218,77791


In [6]:
# create a copy to modify
df_transformed = df.copy()

# dropping from df_transformed columns that are not critical to the ML model

# Assuming df is your DataFrame and columns_to_drop is a list of column names you want to drop
columns_to_drop = ['customer_id', 'zip_code', 'account_id']

# Drop the specified columns
df_transformed = df.drop(columns=columns_to_drop, axis='columns')
df_transformed.head()


,churn_label,gender,age,senior_citizen,married,num_dependents,tenure_months,num_referrals,has_internet_service,internet_type,...,stream_movie,stream_music,total_monthly_fee,total_charges_quarter,total_refunds,area_id,city,latitutde,longitude,population
0,Yes,Female,75,Yes,Yes,0,3,3,Yes,Fiber Optic,...,No,No,83.90,267.40,0.0,607,Camarillo,34.227846,-119.079903,42853
1,No,Male,52,No,Yes,0,63,8,Yes,Fiber Optic,...,No,No,84.65,5377.80,0.0,963,Napa,38.489789,-122.270110,63947
2,No,Female,43,No,Yes,1,65,3,Yes,Cable,...,Yes,Yes,90.45,5957.90,0.0,1390,Sheridan,38.984756,-121.345074,1219
3,No,Female,32,No,No,0,72,0,Yes,Fiber Optic,...,No,No,101.30,7261.25,0.0,303,La Mesa,32.782501,-117.016110,24005
4,No,Female,39,No,No,0,56,0,Yes,DSL,...,No,No,45.05,2560.10,0.0,716,Los Olivos,34.704340,-120.026090,1317


In [7]:
df_transformed.info()
df_transformed.dtypes.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6991 entries, 0 to 6990
Data columns (total 34 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   churn_label                    6991 non-null   object 
 1   gender                         6991 non-null   object 
 2   age                            6991 non-null   int64  
 3   senior_citizen                 6991 non-null   object 
 4   married                        6991 non-null   object 
 5   num_dependents                 6991 non-null   int64  
 6   tenure_months                  6991 non-null   int64  
 7   num_referrals                  6991 non-null   int64  
 8   has_internet_service           6991 non-null   object 
 9   internet_type                  6991 non-null   object 
 10  has_unlimited_data             6991 non-null   object 
 11  has_phone_service              6991 non-null   object 
 12  has_multiple_lines             6991 non-null   o

object     20
int64       7
float64     7
Name: count, dtype: int64

In [8]:
# convert city variable to string
df['city'] = df['city'].astype(str)

In [9]:
# label encoding (binary variables)
label_encoding_columns = ['churn_label', 'gender', 'has_internet_service', 'has_unlimited_data', 'has_phone_service', 'has_multiple_lines', 'has_premium_tech_support','has_online_security', 'has_online_backup', 'has_device_protection', 'paperless_billing', 'stream_tv', 'stream_movie', 'stream_music', 'senior_citizen', 'married']

# encode categorical binary features using label encoding
for column in label_encoding_columns:
    if column == 'gender':
        df_transformed[column] = df_transformed[column].map({'Female': 1, 'Male': 0})
    else: 
        df_transformed[column] = df_transformed[column].map({'Yes': 1, 'No': 0}) 


In [10]:
# one-hot encoding (categorical variables with more than two levels)
one_hot_encoding_columns = ['internet_type', 'contract_type', 'payment_method']

# encode categorical variables with more than two levels using one-hot encoding, dropping the first created column to avoid multicollinearity
df_transformed = pd.get_dummies(df_transformed, columns = one_hot_encoding_columns, dtype=int, drop_first=True)

In [11]:
# min-max normalization (numeric variables)
min_max_columns = ['tenure_months', 'total_monthly_fee', 'total_charges_quarter']

# scale numerical variables using min max scaler
for column in min_max_columns:
        # minimum value of the column
        min_column = df_transformed[column].min()
        # maximum value of the column
        max_column = df_transformed[column].max()
        # min max scaler
        df_transformed[column] = (df_transformed[column] - min_column) / (max_column - min_column)


In [12]:
df_transformed.head()

,churn_label,gender,age,senior_citizen,married,num_dependents,tenure_months,num_referrals,has_internet_service,has_unlimited_data,...,latitutde,longitude,population,internet_type_DSL,internet_type_Fiber Optic,internet_type_None,contract_type_One Year,contract_type_Two Year,payment_method_Credit Card,payment_method_Mailed Check
0,1,1,75,1,1,0,0.028169,3,1,1,...,34.227846,-119.079903,42853,0,1,0,0,0,1,0
1,0,0,52,0,1,0,0.873239,8,1,0,...,38.489789,-122.270110,63947,0,1,0,0,1,1,0
2,0,1,43,0,1,1,0.901408,3,1,1,...,38.984756,-121.345074,1219,0,0,0,0,1,1,0
3,0,1,32,0,0,0,1.000000,0,1,1,...,32.782501,-117.016110,24005,0,1,0,0,1,0,0
4,0,1,39,0,0,0,0.774648,0,1,1,...,34.704340,-120.026090,1317,1,0,0,0,1,0,0


In [13]:
# Add here the ML models tested

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
# Split the transformed data into features and target variable
X = df_transformed.drop(['churn_label','city'], axis=1)
y = df_transformed['churn_label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# XGBoost
import xgboost as xgb

# Create an XGBoost model
model = xgb.XGBClassifier()

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate the AUC score
auc_score = roc_auc_score(y_test, y_pred)
print("XGBoost AUC Score:", auc_score)


XGBoost AUC Score: 0.8045376974224397


In [15]:
# CatBoost
from catboost import CatBoostClassifier
# Create a CatBoost model
model = CatBoostClassifier(verbose=False)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate the AUC score
auc_score = roc_auc_score(y_test, y_pred)
print("CatBoost AUC Score:", auc_score)

CatBoost AUC Score: 0.8182452412001782


In [24]:
# Initialize a LightGBM Classifier with 'auc' as the evaluation metric
lgbm_model = LGBMClassifier(objective='binary', metric='auc', random_state=42)

# Fit the model on the training data
lgbm_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='auc')

# Make predictions on the test set
y_pred = lgbm_model.predict_proba(X_test)[:, 1]

# Calculate the AUC score
auc_score = roc_auc_score(y_test, y_pred)
print("LightGBM AUC Score:", auc_score)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1323, number of negative: 4269
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001709 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2427
[LightGBM] [Info] Number of data points in the train set: 5592, number of used features: 36
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.236588 -> initscore=-1.171478
[LightGBM] [Info] Start training from score -1.171478
LightGBM AUC Score: 0.9219755392054267


In [29]:
# Select the best performing model and input to estimators for RandomizedSearchCV

from sklearn.model_selection import RandomizedSearchCV

# define the parameter grid
grid_parameters = {
    'num_leaves': [20, 31, 40, 50],
    'max_depth': [5, 10, 15, 20, -1],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 300, 400, 500],
    'min_child_samples': [10, 20, 30, 40],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'reg_alpha': [0, 0.01, 0.1, 0.5, 1.0],
    'reg_lambda': [0, 0.01, 0.1, 0.5, 1.0]
}

lgbm_model = LGBMClassifier(objective='binary', metric='auc', random_state=42)

# define the RandomizedSearchCV class for trying different parameter combinations
random_search = RandomizedSearchCV(estimator=lgbm_model,
                                   param_distributions=grid_parameters,
                                   cv=5,
                                   n_iter=150,
                                   n_jobs=-1)

# fitting the model for random search 
random_search.fit(X_train, y_train)

# print best parameter after tuning
print(random_search.best_params_)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1323, number of negative: 4269
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003273 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2427
[LightGBM] [Info] Number of data points in the train set: 5592, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.236588 -> initscore=-1.171478
[LightGBM] [Info] Start training from score -1.171478
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

In [36]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# make the predictions
random_search_predictions = random_search.predict_proba(X_test)[:, 1]

In [38]:
# print the auc score of the model after hyparameter tuning 
final_model_score = roc_auc_score(y_test, random_search_predictions)
print(f"The final model score is {final_model_score}")

The final model score is 0.9232635143475064


In [35]:
# Initialize a LightGBM Classifier with 'auc' as the evaluation metric
lgbm_model = LGBMClassifier(objective='binary', metric='auc', random_state=42, subsample= 0.9, reg_lambda=1.0, reg_alpha= 0.5, num_leaves=40, n_estimators=200, min_child_samples=40, max_depth=10, learning_rate=0.05, colsample_bytree=0.6)

# Fit the model on the training data
lgbm_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='auc')

# Make predictions on the test set
y_pred = lgbm_model.predict_proba(X_test)[:, 1]

# Calculate the AUC score
auc_score = roc_auc_score(y_test, y_pred)
print("LightGBM AUC Score:", auc_score)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1323, number of negative: 4269
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002342 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2427
[LightGBM] [Info] Number of data points in the train set: 5592, number of used features: 36
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.236588 -> initscore=-1.171478
[LightGBM] [Info] Start training from score -1.171478
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

In [39]:
pip install joblib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from joblib import dump

# Export the trained model to a .pkl file
dump(random_search, 'random_search_model.pkl')

# You can also specify a directory path to save the file to a specific location
# For example:
# dump(random_search, '/path/to/save/random_search_model.pkl')
